In [1]:
import pandas as pd
import numpy as np

import requests

import os
import time
from datetime import datetime
import sys
import yaml
import gc


from data_loader import *
from utils import *
from nn import *


import seaborn as sns
sns.set_style("darkgrid")
plt.rcParams.update({
    'axes.facecolor': '#1e1e1e',
    'figure.facecolor': '#1e1e1e',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'white',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'axes.grid': True,
    'grid.color': 'gray'
})

pd.set_option("display.max_column",None)
print(os.getcwd())


%load_ext autoreload
%reload_ext autoreload
%autoreload 2


def auto_reload():
    %load_ext autoreload
    %reload_ext autoreload
    %autoreload 2

/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app


In [2]:
with open("config.yaml",'r') as f:
    config = yaml.safe_load(f)

home_dir = config['HOME_DIRECTORY']
home_dir

'/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG'

In [3]:
# Model File Configs:
# Every Run of the notebook is logged in to a submodel folder for it

############################################################################
# Please set output path to the project directory where it is uncompressed #
############################################################################

project_path = home_dir + "/outputs/models/"




output_path = ""

submodel_name = ""

encoding_path = ""
feature_report_path = ""
def create_submodel(model_name:str):
    author = "EJ"
    global submodel_name
    submodel_name = datetime.now().strftime("%d_%H_%M") + "_"+model_name
    global output_path
    output_path = project_path+submodel_name
    global encoding_path
    encoding_path = output_path+"/encodings/"
    global feature_report_path
    feature_report_path = output_path+"/feature_report/"
    os.mkdir(output_path)
    os.mkdir(encoding_path)
    os.mkdir(feature_report_path)


In [4]:
create_submodel("test")

# API Football:
https://www.api-football.com

![PYTHON LOGO](https://www.api-football.com/public/img/news/archi-beta.jpg)

In [5]:
leagues_dat = get_leagues(home_dir +"/data/Leagues/leagues.parquet")
leagues_dat[['league_id','league_name','country_name']]

,league_id,league_name,country_name
0,4,Euro Championship,World
1,21,Confederations Cup,World
2,61,Ligue 1,France
3,144,Jupiler Pro League,Belgium
4,71,Serie A,Brazil
...,...,...,...
1128,1131,Super Cup,Mongolia
1129,1132,AFC Challenge League,World
1130,1133,Goiano - 3,Brazil
1131,1001,CONCACAF Women U20,World


# Leagues subset:

In [6]:
# Configs
major_leagues = ["Premier League","La Liga","Serie A","Bundesliga","Eredivisie","Ligue 1"]
major_countries = ["England","Spain","Italy","Germany","Netherlands","France","Brazil"]
teams = ["Liverpool","Wolves"] # teams to pull players data of
seasons = [2022,2021,2023,2024] # seasons to pull players and teams stats of



leagues_subset = leagues_dat[leagues_dat.league_name.isin(major_leagues) & leagues_dat.country_name.isin(major_countries)] # league ID to pull from, current values: {39:premier league}, Add to dictionary as needed

In [7]:
leagues_subset

,seasons,league_id,league_name,league_type,league_logo,country_name,country_code,country_flag
2,"[{'coverage': {'fixtures': {'events': True, 'l...",61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,https://media.api-sports.io/flags/fr.svg
4,"[{'coverage': {'fixtures': {'events': True, 'l...",71,Serie A,League,https://media.api-sports.io/football/leagues/7...,Brazil,BR,https://media.api-sports.io/flags/br.svg
5,"[{'coverage': {'fixtures': {'events': True, 'l...",39,Premier League,League,https://media.api-sports.io/football/leagues/3...,England,GB,https://media.api-sports.io/flags/gb.svg
6,"[{'coverage': {'fixtures': {'events': True, 'l...",78,Bundesliga,League,https://media.api-sports.io/football/leagues/7...,Germany,DE,https://media.api-sports.io/flags/de.svg
7,"[{'coverage': {'fixtures': {'events': True, 'l...",135,Serie A,League,https://media.api-sports.io/football/leagues/1...,Italy,IT,https://media.api-sports.io/flags/it.svg
8,"[{'coverage': {'fixtures': {'events': True, 'l...",88,Eredivisie,League,https://media.api-sports.io/football/leagues/8...,Netherlands,NL,https://media.api-sports.io/flags/nl.svg
10,"[{'coverage': {'fixtures': {'events': True, 'l...",140,La Liga,League,https://media.api-sports.io/football/leagues/1...,Spain,ES,https://media.api-sports.io/flags/es.svg


# Read All fixtures data

In [8]:
teams_dat = pd.read_parquet(home_dir + "/data/Teams/team_league.parquet")

In [ ]:
fixtures_dir = home_dir + "/data/Fixtures"

complete_data = pd.DataFrame()
for file in os.listdir(fixtures_dir):
    dat = pd.read_parquet(os.path.join(fixtures_dir,file))
    complete_data = pd.concat([complete_data,dat],axis = 0)

complete_data = complete_data.reset_index()
complete_data.drop(columns = ['index'],inplace=True)

/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_63254/3256445224.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  complete_data = pd.concat([complete_data,dat],axis = 0)
/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_63254/3256445224.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  complete_data = pd.concat([complete_data,dat],axis = 0)
/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_63254/3256445224.py:6: FutureWarning: The behavior of DataFrame concatena

In [10]:
complete_data.shape

(148822, 50)

In [11]:
# Data checks
complete_data['passes_accuracy'] = complete_data['passes_accuracy'].astype("float64")

# Targets
complete_data['outcome_num'] = pd.Categorical(complete_data.outcome).codes

complete_data['win'] = np.where(complete_data.outcome.str.lower() == 'win', 1,0)


In [14]:
# This is the dictionary that contains all information about the features    
dat_dict = find_data_types(complete_data,config['OUTCOME_COLS'] + ['outcome_num','outcome'])
dat_dict = pd.DataFrame(list(dat_dict.items()),columns =['feature','type'])

# differentiate modeling features
non_modeling_features = config['FIXTURE_COLS'] + config['OUTCOME_COLS'] + config['MISC_COLS'] + ['outcome_num','win']
dat_dict['modeling_feature'] = np.where(dat_dict['feature'].isin(non_modeling_features),0,1)
dat_dict['encoded'] = 0

print(dat_dict['type'].value_counts())
dat_dict.reset_index(drop= True)


## Encode Features
dat_dict = create_data_index(complete_data,dat_dict,'target',encoding_path)
dat_dict[dat_dict.modeling_feature ==1]

type
numeric        36
target         10
categorical     4
other           2
Name: count, dtype: int64


,feature,type,modeling_feature,encoded
2,offsides,numeric,1,0
9,shots_total,numeric,1,0
10,shots_on,numeric,1,0
11,goals_total,numeric,1,0
12,goals_conceded,numeric,1,0
13,goals_assists,numeric,1,0
14,goals_saves,numeric,1,0
15,passes_total,numeric,1,0
16,passes_key,numeric,1,0
17,passes_accuracy,numeric,1,0


In [ ]:
# primary position map:
player_position = complete_data.groupby(["player_id","games_position"],as_index = False).agg(games_played = ("player_id","size"))
player_position['multiple_records'] = player_position.groupby('player_id')['games_played'].transform("cumsum")
player_position['multiple_records'] = player_position.groupby('player_id')['multiple_records'].transform("max")
player_position['major_position'] = np.where(player_position.games_played/player_position.multiple_records >= .5, player_position.games_position,None)
player_position_map = player_position[['player_id','major_position']].dropna().drop_duplicates()
player_position_map

# Join back to complete_data

complete_data = pd.merge(complete_data,player_position_map,on = 'player_id',how = 'left')

In [16]:
pd.crosstab(complete_data['games_position'], complete_data['major_position'],normalize=True)

major_position,D,F,G,M
games_position,,,,
D,0.308079,0.000309,0.00000,0.010172
F,0.000429,0.188401,0.00000,0.024632
G,0.000000,0.000000,0.11115,0.000000
M,0.015238,0.022995,0.00000,0.318594


In [55]:
# quick knn clustering
from sklearn.cluster import KMeans

# who is ekitike most similar to 

interest_col_set = list(set(['major_position','player_id','player_name'] + config["ATTACK_COLS"] +  config["PASSING_COLS"]))


# primary position:



striker_stats = complete_data[interest_col_set].fillna(0)[complete_data[interest_col_set].fillna(0).major_position.isin(['F'])].groupby(['player_id','player_name'],as_index = False).agg(n_games = ('player_name','size'),
                                                                          avg_dribble_success = ('dribble_success_rate','mean'),
                                                                          std_dribble_success = ('dribble_success_rate','std'),
                                                                          avg_target_shot_cr = ('target_shot_conversion_perc','mean'),
                                                                          std_target_shot_cr = ('target_shot_conversion_perc','std'),
                                                                          avg_duels_cr = ('duels_won_perc','mean'),
                                                                          std_duels_cr = ('duels_won_perc','std'),
                                                                          avg_pass_accuracy = ('pass_accuracy_perc','mean'),
                                                                          std_pass_accuracy = ('pass_accuracy_perc','std')
                                                                          )


striker_stats = striker_stats[striker_stats.n_games > 40]
striker_stats.shape


(244, 11)

In [43]:
striker_stats.sort_values('avg_dribble_success',ascending=False)

,player_id,player_name,n_games,avg_dribble_success,std_dribble_success,avg_target_shot_cr,std_target_shot_cr,avg_duels_cr,std_duels_cr,avg_pass_accuracy,std_pass_accuracy
363,2298,Callum Hudson-Odoi,62,54.735372,41.768795,15.322581,33.374072,50.344991,29.580075,83.607772,18.026481
1036,30937,Mattia Zaccagni,62,53.106759,36.390104,20.698925,39.886620,57.121493,17.837042,83.857680,7.540389
2390,350927,Henry Mosquera,57,51.857738,30.645705,2.631579,14.702565,42.199637,21.685586,78.338753,19.491532
396,2612,Giorgian De Arrascaeta,45,51.746032,44.499119,21.851852,39.058074,50.238737,23.200553,77.182652,18.310961
376,2454,Yeferson Soteldo,46,51.617495,27.608217,13.043478,34.050261,48.971655,14.895668,82.430937,8.556826
...,...,...,...,...,...,...,...,...,...,...,...
2625,395761,Mathis Lambourde,41,2.439024,15.617376,2.439024,15.617376,7.344173,19.850721,11.368564,26.526279
2749,416112,Paulo Vitor,50,2.000000,14.142136,0.000000,0.000000,13.146032,23.272039,20.142375,35.776515
942,25400,Yannick Gerhardt,61,1.639344,12.803688,4.918033,21.803895,39.376914,28.954522,65.440745,30.142649
2138,319919,Oussama El Azzouzi,47,0.000000,0.000000,4.255319,20.402971,14.463019,28.270040,37.406482,44.534061


# Clustering

In [56]:
# Fit all players within 20 buckets
n_clusters = 20


km = KMeans(n_clusters,random_state= 5)
striker_stats['cluster_dribble'] = km.fit_predict(striker_stats[['avg_dribble_success']])

km = KMeans(n_clusters,random_state= 5)
striker_stats['cluster_shot_cr'] = km.fit_predict(striker_stats[['avg_target_shot_cr']])

km = KMeans(n_clusters,random_state= 5)
striker_stats['cluster_duels'] = km.fit_predict(striker_stats[['avg_duels_cr']])

km = KMeans(n_clusters,random_state= 5)
striker_stats['cluster_pass'] = km.fit_predict(striker_stats[['avg_pass_accuracy']])

In [ ]:
# Best Goal Scorers:
striker_stats.sort_values(["avg_target_shot_cr","cluster_shot_cr"],ascending= [0,0])[["player_name","n_games","avg_target_shot_cr","std_target_shot_cr","cluster_shot_cr"]].head(15)

,player_name,n_games,avg_target_shot_cr,std_target_shot_cr,cluster_shot_cr
845,Serhou Guirassy,58,47.586207,43.491807,13
78,Mohamed Salah,70,38.214286,41.545545,6
419,Alexander Isak,65,37.512821,42.151474,6
108,Robert Lewandowski,71,35.727700,42.194497,15
501,Alexander Sørloth,72,35.300926,44.460183,15
738,Chris Wood,67,34.701493,44.890369,15
71,Kylian Mbappé,99,34.360269,35.965550,15
279,Alexandre Lacazette,59,32.937853,41.524284,16
683,Artem Dovbyk,71,32.746479,43.559937,16
197,Romelu Lukaku,68,31.862745,43.740462,16


In [47]:
striker_stats[striker_stats.cluster_shot_cr == 6]

,player_id,player_name,n_games,avg_dribble_success,std_dribble_success,avg_target_shot_cr,std_target_shot_cr,avg_duels_cr,std_duels_cr,avg_pass_accuracy,std_pass_accuracy,cluster_dribble,cluster_shot_cr,cluster_duels,cluster_pass
78,306,Mohamed Salah,70,36.143476,32.717543,38.214286,41.545545,36.406991,16.523151,74.740807,8.777041,12,6,17,6
419,2864,Alexander Isak,65,41.008547,37.573602,37.512821,42.151474,33.051244,20.340532,77.344333,12.969312,8,6,12,14


In [49]:
# Best Dribblers:
striker_stats.sort_values(["avg_dribble_success","cluster_dribble"],ascending= [0,0])[["player_name","n_games","avg_dribble_success","std_dribble_success","cluster_dribble"]].head(20)

,player_name,n_games,avg_dribble_success,std_dribble_success,cluster_dribble
363,Callum Hudson-Odoi,62,54.735372,41.768795,6
1036,Mattia Zaccagni,62,53.106759,36.390104,6
2390,Henry Mosquera,57,51.857738,30.645705,6
396,Giorgian De Arrascaeta,45,51.746032,44.499119,6
376,Yeferson Soteldo,46,51.617495,27.608217,6
690,Mohammed Kudus,65,51.513547,23.010520,6
1550,Rayan Cherki,64,51.182416,36.574986,6
267,Jérémy Doku,64,51.144831,35.173646,6
839,Rayan Aït Nouri,70,50.780743,33.055679,6
411,Moses Simon,54,50.558161,30.666850,6


In [51]:
striker_stats[striker_stats.cluster_dribble == 6]

,player_id,player_name,n_games,avg_dribble_success,std_dribble_success,avg_target_shot_cr,std_target_shot_cr,avg_duels_cr,std_duels_cr,avg_pass_accuracy,std_pass_accuracy,cluster_dribble,cluster_shot_cr,cluster_duels,cluster_pass
218,987,Paulinho,62,50.215054,34.532778,29.838710,41.831597,45.011072,20.906516,84.773035,7.515955,6,19,8,9
267,1422,Jérémy Doku,64,51.144831,35.173646,7.812500,25.539807,48.640562,25.245109,76.636649,26.415751,6,8,1,14
363,2298,Callum Hudson-Odoi,62,54.735372,41.768795,15.322581,33.374072,50.344991,29.580075,83.607772,18.026481,6,1,14,9
376,2454,Yeferson Soteldo,46,51.617495,27.608217,13.043478,34.050261,48.971655,14.895668,82.430937,8.556826,6,10,1,2
396,2612,Giorgian De Arrascaeta,45,51.746032,44.499119,21.851852,39.058074,50.238737,23.200553,77.182652,18.310961,6,5,14,14
411,2781,Moses Simon,54,50.558161,30.666850,16.358025,34.793602,44.250778,17.350623,75.623057,11.985077,6,1,19,6
603,10135,Bruno Guimarães,75,49.637037,40.696462,13.333333,33.220530,54.976542,14.684849,83.651523,6.029310,6,10,13,9
668,13708,Jhon Arias,57,50.146841,31.996314,16.959064,34.285685,46.642049,15.522593,86.691166,13.556094,6,1,11,17
690,15911,Mohammed Kudus,65,51.513547,23.010520,15.384615,34.992272,47.676258,17.170001,85.883882,13.916729,6,1,11,17
789,19586,Eberechi Eze,61,49.550008,30.452032,19.398907,35.639700,48.175339,16.886014,81.348240,10.022810,6,11,1,2


In [57]:
striker_stats.sort_values(["avg_duels_cr","cluster_duels"],ascending= [0,0])[["player_name","n_games","avg_duels_cr","std_duels_cr","cluster_duels"]].head(20)

,player_name,n_games,avg_duels_cr,std_duels_cr,cluster_duels
583,Derik Lacerda,47,56.183757,22.734630,7
1027,Lamine Yamal,73,55.038893,17.504645,7
212,Deyverson,58,53.884131,20.213160,7
308,Florian Sotoca,63,52.049234,19.635543,11
326,Luiz Araújo,46,50.899797,24.423816,11
905,Kenan Yıldız,54,50.648629,23.205736,11
313,Serhou Guirassy,58,50.631990,18.415577,11
577,Anthony Gordon,69,50.017103,19.037572,11
343,Wout Weghorst,54,48.824008,26.557223,19
92,Jérémy Doku,64,48.640562,25.245109,19


In [27]:
auto_reload()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
find_player(data = complete_data,player_name="Sal")

,player_id,player_name,team
9390,162451,Anass Salah-Eddine,Twente
11855,26252,Roland Sallai,SC Freiburg
17512,22090,William Saliba,Arsenal
19865,47480,Mohammed Salisu,Monaco
29677,352990,Amine Salama,Reims
30414,375000,Ibrahim Salah,Stade Brestois 29
32887,24807,Salih Özcan,Borussia Dortmund
40609,462277,Iliesse Salhi,Nice
52893,306,Mohamed Salah,Liverpool
66117,375000,Ibrahim Salah,Rennes


In [29]:
striker_stats.sort_values(["avg_pass_accuracy","cluster_pass"],ascending= [0,0])[["player_name","n_games","avg_pass_accuracy","std_pass_accuracy","cluster_pass"]].head(15)

,player_name,n_games,avg_pass_accuracy,std_pass_accuracy,cluster_pass
74,Paulinho,62,84.773035,7.515955,17
16,Heung-min Son,65,84.743458,5.641457,17
265,Iliman Ndiaye,63,84.485700,9.120671,17
222,Rodrygo,68,84.274770,22.102794,17
183,Jonathan David,66,83.094075,9.994756,17
110,Pedro Neto,57,83.076093,18.170627,17
93,Bukayo Saka,60,82.859731,8.506756,17
39,Khvicha Kvaratskhelia,67,82.713524,16.861145,17
682,Florian Wirtz,63,82.494204,8.341682,17
31,Kylian Mbappé,99,82.365391,13.634436,17


In [30]:
striker_stats[striker_stats.cluster_pass == 4]

,player_id,player_name,n_games,avg_dribble_success,std_dribble_success,avg_target_shot_cr,std_target_shot_cr,avg_duels_cr,std_duels_cr,avg_pass_accuracy,std_pass_accuracy,cluster_dribble,cluster_shot_cr,cluster_duels,cluster_pass
21,219,Matteo Politano,74,34.369369,37.510732,14.189189,34.641283,41.764276,23.246070,81.186237,8.298858,5,9,17,4
61,842,Nikola Vlašić,64,39.973958,41.525679,10.156250,28.423788,43.593763,24.119208,81.013991,14.442045,0,7,13,4
80,1165,Matheus Cunha,65,45.617216,28.412309,24.743590,38.329065,45.204846,18.434670,80.714919,8.293235,4,14,8,4
132,2489,Luis Díaz,75,44.663492,36.692760,15.977778,31.663221,35.738410,18.921356,80.577637,19.303255,13,15,15,4
166,6009,Julián Álvarez,75,29.288889,40.047057,25.111111,40.280858,27.700026,21.237165,80.936999,17.735340,18,14,18,4
296,19586,Eberechi Eze,61,49.550008,30.452032,19.398907,35.639700,48.175339,16.886014,81.348240,10.022810,11,16,19,4
369,30531,Jérémie Boga,52,43.289072,33.653019,8.653846,25.669761,46.888627,26.394004,80.663274,21.706151,13,18,2,4
516,85041,Amine Gouiri,66,39.183502,38.647753,18.686869,36.545856,37.604319,24.224621,79.761278,17.584616,0,16,6,4
577,138787,Anthony Gordon,69,37.528180,35.187186,19.806763,37.407332,50.017103,19.037572,81.466326,9.428434,9,16,11,4
905,339883,Kenan Yıldız,54,49.885362,35.762224,12.037037,32.118861,50.648629,23.205736,82.072258,14.765685,11,2,11,4


In [ ]:
filter_query = 'major_position.isin(["F"])'


# Stat to look at:
stat = 'target_shot_conversion_perc'
agg_fun = "mean"
rank_cutoff = 20

# configs 
min_appearance = 40

dribble_dat_g = complete_data.query(filter_query).reset_index().fillna(0).groupby("player_name").agg(n_apps = ("player_name","size"),stat = (stat,agg_fun)).reset_index()
dribble_dat_g = dribble_dat_g[dribble_dat_g.n_apps >= min_appearance]
dribble_dat_g['rank'] = dribble_dat_g["stat"].fillna(0).rank(ascending= False,method = 'dense')
dribble_dat_g.sort_values("rank",inplace = True)

fig, ax = plt.subplots(figsize=(13, 8))

# Plot correctly, no comma here
sns.boxplot(
    data=complete_data.query(filter_query)[complete_data.query(filter_query).player_name.isin(dribble_dat_g[dribble_dat_g['rank'] < rank_cutoff]['player_name'])],
    x="player_name",
    y=stat,
    order=dribble_dat_g[dribble_dat_g['rank'] < rank_cutoff]['player_name'],
    ax=ax,
    
)

# Now this works correctly on `ax`
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_title(f"Stat: {stat}")
plt.tight_layout()
plt.show()


In [ ]:
dribble_dat_g

In [ ]:
striker_stats[striker_stats.player_name.str.contains("Adam")]

In [ ]:
stat

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (12,8))

sns.boxplot(complete_data[complete_data.player_name == 'Adam Hložek'],x = 'player_name', y = "target_shot_conversion_perc",ax = ax)

In [ ]:
complete_data[complete_data.player_name == 'Adam Hložek'][["player_name",stat] + config["ATTACK_COLS"]]

In [ ]:
fig = plt.subplots(nrows=1, ncols = 1, figsize = (20,10))
fig = sns.heatmap(complete_data.query(filter_query)[config['ATTACK_COLS'] + ['team_goals_scored','team_non_penalty_goals_scored','team_goals_conceded']].corr(),cmap = 'coolwarm')
fig.set_xticklabels(fig.get_xticklabels(),rotation = 60)


In [ ]:
# trial multiclass model:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(complete_data[list(set(config['DEFENSE_COLS'] + config['PASSING_COLS'] + config['ATTACK_COLS'])) + ['outcome_num']].drop(columns = 'outcome_num'),complete_data[list(set(config['DEFENSE_COLS'] + config['PASSING_COLS'] + config['ATTACK_COLS'])) + ['outcome_num']][['outcome_num']],stratify=nn_dat[['outcome_num']],random_state=33)


In [ ]:
dat = NNDataFromPd(X_train.fillna(0), y_train.outcome_num, dat_dict)
train_loader = DataLoader(dat, batch_size = 128,shuffle= True)

In [ ]:
train_loader.dataset.X_numeric_tensor.shape

In [ ]:
# model params
n_features = X_train.shape[1]
n_classes = y_train.iloc[:,0].nunique()
model = MultiClassModel(n_features,n_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)

In [ ]:
epochs = 500

for epoch in range(epochs):
    
    epoch_loss = 0

    for X_numeric_batch, X_categoric_batch, y_batch in train_loader:
        
        pred = model.forward(X_numeric_batch)
        
        loss = criterion(pred,y_batch)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch: {epoch}, Loss: {epoch_loss}")

In [ ]:
model.eval()

In [ ]:
X_test['passes_accuracy'] = X_test['passes_accuracy'].astype("float64")

In [ ]:
# test sets

test_dat = NNDataFromPd(X_test,y_test,dat_dict)
test_loader = DataLoader(test_dat,batch_size= X_test.shape[0],shuffle=True)

In [ ]:
model.eval()
with torch.no_grad():
    for X_numeric_batch, X_categoric_batch, y_batch in test_loader:
        output = model(X_numeric_batch)
        pred_class = torch.argmax(output, dim = 1)

In [ ]:
# Logistic Model:
X_train, X_test, y_train, y_test = train_test_split(complete_data[complete_data.games_position == 'F'][list(set(config['DEFENSE_COLS'] + config['PASSING_COLS'] + config['ATTACK_COLS'])) + ['win']].drop(columns = 'win'),
                                                    complete_data[complete_data.games_position == 'F']['win'],
                                                    stratify=complete_data['win'],
                                                    random_state=33)

In [ ]:
train_dat = NNDataFromPd(X_train,y_train,dat_dict)
train_loader = DataLoader(train_dat,batch_size= 128,shuffle = True)

In [ ]:
n_features = X_train.shape[1]
model = LogisticNNModelComplex(n_features)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.005)

In [ ]:
epochs = 500
for epoch in range(epochs):
    epoch_loss = 0
    
    for X_numeric, X_categoric, y in train_loader:

        pred = model(X_numeric)

        loss = criterion(pred,y.unsqueeze(1))

        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch},  Loss: {loss}")


In [ ]:
pred_proba.squeeze(1)

In [ ]:
from validations import *


test_dat = NNDataFromPd(X_test,y_test,dat_dict)
test_loader = DataLoader(test_dat,batch_size= X_test.shape[0],shuffle=True)

model.eval()
with torch.no_grad():
    for X_numeric_batch, X_categoric_batch, y_batch in test_loader:
        output = model(X_numeric_batch)
        pred_proba = torch.softmax(output,dim =1)
        pred_class = torch.argmax(output, dim = 1)


discrete_evaluations(y_test,pred_class,pred_proba.squeeze(1),classification_type="Binary",model_path= output_path)